<a href="https://colab.research.google.com/github/adamrhurley/AI_Assignment2.2/blob/main/MusicRecommenderSystem/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Reference: https://www.kaggle.com/mcarujo/knn-x-autoencoder-movie-recommendation

In [ ]:
import pandas as pd
import numpy as np
import io

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving artists.csv to artists (1).csv
Saving userArtists.csv to userArtists (1).csv


In [ ]:
userArtists = pd.read_csv(io.BytesIO(uploaded['userArtists.csv']))
artists = pd.read_csv(io.BytesIO(uploaded['artists.csv']))

In [ ]:
data = pd.merge(userArtists, artists, on='artistID')

In [ ]:
userArtists.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [ ]:

artists['total_plays'] = userArtists.weight.value_counts()
artists['mean_plays'] = userArtists.groupby('artistID').mean()['weight']
artists.head(30)

,artistID,name,url,pictureURL,total_plays,mean_plays
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg,NaN,NaN
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg,636.0,257.000000
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...,419.0,667.666667
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...,338.0,258.333333
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...,345.0,281.500000
5,6,Moonspell,http://www.last.fm/music/Moonspell,http://userserve-ak.last.fm/serve/252/2181591.jpg,318.0,456.500000
6,7,Marilyn Manson,http://www.last.fm/music/Marilyn+Manson,http://userserve-ak.last.fm/serve/252/2558217.jpg,311.0,508.000000
7,8,DIR EN GREY,http://www.last.fm/music/DIR+EN+GREY,http://userserve-ak.last.fm/serve/252/46968835...,281.0,723.315789
8,9,Combichrist,http://www.last.fm/music/Combichrist,http://userserve-ak.last.fm/serve/252/51273485...,262.0,899.250000
9,10,Grendel,http://www.last.fm/music/Grendel,http://userserve-ak.last.fm/serve/252/5872875.jpg,295.0,400.875000


In [ ]:
print(artists.loc[artists['name'] == 'Martin Solveig'])

      artistID            name  ... total_plays mean_plays
1394      1403  Martin Solveig  ...        11.0  504.02381

[1 rows x 6 columns]


In [ ]:
import plotly.express as px
data = px.data.gapminder()

fig = px.bar(
    artists[:10].sort_values('total_plays', ascending=False),
    x='name',
    y='total_plays',
    color='mean_plays',
    height=600,
    title='Top 10 - Popular artists',
    labels={'total_plays':'Total number of plays','mean_rating': 'Mean plays', 'name': 'Name'}
)
fig.show()

In [ ]:
def distance(x, y):
    return np.linalg.norm(x - y)

def ratings_by_user(user_id):
    user_r =userArtists[ userArtists.userID == user_id ]
    return user_r.drop(['userID'], axis=1)


def merge(df1,df2):
    return df1.merge(df2, how='inner', on="artistID")

# to calc the distance of two users
def distance_between_users(user_id1, user_id2):
    vector_user1 = ratings_by_user(user_id1)
    vector_user2 = ratings_by_user(user_id2)
    vectors = merge(vector_user1, vector_user2)
    
    if vectors.empty:
        return (user_id1, user_id2, 100)
    return user_id1, user_id2, distance(vectors['weight_x'], vectors['weight_y'])

# get all users ids...
def get_users_ids():
    return list(userArtists.userID.unique())


def distance_from_all(user_id):
    if not user_id in userArtists.userID:
        return False   
    
    ids = get_users_ids()
    #ids.remove(user_id)
    return [distance_between_users(user_id, for_id) for for_id in ids[:10]]

In [ ]:

def artists_recommend(userID, k=10):
    points = distance_from_all(userID)
    points = pd.DataFrame(points, columns=['myId','userID','distance'])
    points.sort_values('distance', axis=0, ascending=True, inplace=True)
    artist_list = list()

    my_ratings = ratings_by_user(userID)
    our_users = list(points.userID)

    for our_user in our_users:
        merged = my_ratings.merge(ratings_by_user(our_user), how='right', on="artistID")
        merged = merged[merged.weight_x.isna()]
        artist_list.append(merged)
        artist_list = pd.concat(artist_list)
        return artists.loc[artist_list.artistID].sort_values('mean_plays', ascending=False)[:k]
        

In [ ]:
artists_recommend(114)

,artistID,name,url,pictureURL,total_plays,mean_plays
279,285,Janet Jackson,http://www.last.fm/music/Janet+Jackson,http://userserve-ak.last.fm/serve/252/4245358.jpg,101.0,2022.540541
257,263,Jill Scott,http://www.last.fm/music/Jill+Scott,http://userserve-ak.last.fm/serve/252/61389167...,113.0,1594.634855
285,291,Kelly Clarkson,http://www.last.fm/music/Kelly+Clarkson,http://userserve-ak.last.fm/serve/252/28236733...,110.0,1325.989011
287,293,Ashlee Simpson,http://www.last.fm/music/Ashlee+Simpson,http://userserve-ak.last.fm/serve/252/33543587...,105.0,1078.750000
239,245,Bob Marley & The Wailers,http://www.last.fm/music/Bob%2BMarley%2B%2526%...,http://userserve-ak.last.fm/serve/252/208159.jpg,124.0,1018.000000
265,271,Mos Def,http://www.last.fm/music/Mos+Def,http://userserve-ak.last.fm/serve/252/28041091...,110.0,1016.743902
267,273,Avant,http://www.last.fm/music/Avant,http://userserve-ak.last.fm/serve/252/135431.jpg,129.0,876.000000
262,268,Richard Marx,http://www.last.fm/music/Richard+Marx,http://userserve-ak.last.fm/serve/252/7905405.jpg,101.0,836.545455
276,282,Boss AC,http://www.last.fm/music/Boss+AC,http://userserve-ak.last.fm/serve/252/46755589...,129.0,689.428571
246,252,Tamia,http://www.last.fm/music/Tamia,http://userserve-ak.last.fm/serve/252/26185453...,118.0,653.142857
